In [1]:
#dependencies and point to where the data will live
import json
import pandas as pd
import numpy as np

file_dir = "C://Users/ryang/Documents/Bootcamp/Module8-ETL/movies-etl/Data/"

In [2]:
# load json data
with open(f'{file_dir}wikipedia-movies.json', mode = 'r') as file:
    wiki_movies_raw = json.load(file)

In [3]:
#load kaggle data
kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir}ratings.csv')

In [4]:
#check out wiki data

wiki_movies_df = pd.DataFrame(wiki_movies_raw)

len(wiki_movies_df.columns)

193

In [5]:
#filter out movies without a director or imdb link and add tv

wiki_movies = [movie for movie in wiki_movies_raw
                if ('Director' in movie or 'Directed by' in movie)
                and 'imdb_link' in movie
                and 'No. of episodes' not in movie]

len(wiki_movies)

7076

In [6]:
#starting the clean movie function

def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    
    #take all alternative titles, collapse to one column
    alt_titles = {}

    #loop through alt title names
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:

        #if any alt titles exist, add them to a dictionary, put that dictionary in movie list
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
        
        if len(alt_titles) > 0:
            movie['alt_titles'] = alt_titles

    return movie

In [7]:
clean_movies = [clean_movie(movie) for movie in wiki_movies]

len(clean_movies)

7076

In [9]:
wiki_movies_df = pd.DataFrame(clean_movies)

wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Preceded by',
 'Adaptation by',
 'Suggested by',
 'alt_titles',
 'Released',
 'Recorded',
 'Venue',
 'Length',
 'Label',
 'Director',
 'Producer',
 'Animation by',
 'Color process',
 'Screen story by']